In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from src.data.make_dataset import df

In [2]:
df.head()

,Starttid,Sluttid,Navn,Rolle,Tekst,Dagsordenpunkt,Parti
0,2009-10-06 12:00:32.623,2009-10-06 12:02:08.360,Niels Helveg Petersen,aldersformanden,Mødet er åbnet.I henhold til grundloven er Fol...,Punkt 0,NaN
1,2009-10-06 12:02:08.360,2009-10-06 12:04:01.713,None,formand,"Jeg vil gerne takke for den tillid, som Tinget...",Punkt 0,NaN
2,2009-10-06 12:04:01.713,2009-10-06 12:11:14.713,Lars Løkke Rasmussen,minister,Danmark er et godt land at leve i. Vi har bygg...,Punkt 0,NaN
3,2009-10-06 12:11:14.713,2009-10-06 12:14:49.713,Lars Løkke Rasmussen,minister,"Det er ikke den eneste mulighed, vi skal gribe...",Punkt 0,NaN
4,2009-10-06 12:14:49.713,2009-10-06 12:19:47.713,Lars Løkke Rasmussen,minister,For det første – og helt afgørende: Vi skal ho...,Punkt 0,NaN


In [10]:
feature_word = [word for word, count in sorted(vocabulary.items(), key = lambda kv:(kv[1], kv[0]))]

In [3]:
X

<160444x23574 sparse matrix of type '<class 'numpy.int64'>'
	with 9014135 stored elements in Compressed Sparse Row format>